In [1]:
sps_path = input("Where are the SPS files? ")

Where are the SPS files? Desktop\SPS Test\SPS


In [2]:
rps_path = input("Where are the RPS files? ")

Where are the RPS files? Desktop\SPS Test\RPS


In [3]:
output_path = input("Where shall I put the output? ")

Where shall I put the output? Desktop\SPS Test


In [4]:
#Get the tools required
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

from geopandas import GeoDataFrame
from shapely.geometry import Point
from shapely.geometry import LineString


#Grab filenames
sps_files = glob.glob(os.environ['USERPROFILE']+"\\"+sps_path+"\*.s01")
rps_files = glob.glob(os.environ['USERPROFILE']+"\\"+rps_path+"\*.r01")


In [5]:
#Set column widths for SPS
colspecs=[[0,9], [14,21], [23,27], [31,34], [37,38], [41,46], [47,55],[55,65], [68,71], [71,87]]
names=['Line Number', 'Shot Number', 'Attempt', 'Gun Depth', 'Num', 'Depth', 'Easting', 'Northing', 'Geodetic Height', 'Time']

#Create a dataframe for the SPS
all_sps = pd.DataFrame()

for i in sps_files:
    df = pd.read_fwf(i,colspecs=colspecs,names=names)
    
    df = df.loc[df["Line Number"].str.startswith("S")]
    all_sps = all_sps.append(df) 

#Convert E/N/D to numbers    
all_sps["Easting"] = pd.to_numeric(all_sps["Easting"])
all_sps["Northing"] = pd.to_numeric(all_sps["Northing"])
all_sps["Depth"] = pd.to_numeric(all_sps["Depth"])

#Check
all_sps.head()

,Line Number,Shot Number,Attempt,Gun Depth,Num,Depth,Easting,Northing,Geodetic Height,Time
130,S11779001,2143.00,1A1,4.0,0,128.3,466667.6,6567071.6,0.8,157015940.572000
131,S11779001,2142.00,1A2,4.0,0,128.7,466684.6,6567047.8,0.8,157015944.908000
132,S11779001,2141.00,1A1,4.0,0,128.1,466655.1,6567049.6,0.8,157015951.152000
133,S11779001,2140.00,1A2,4.0,0,128.8,466671.6,6567025.0,0.8,157015955.880000
134,S11779001,2139.00,1A1,4.0,0,127.8,466642.6,6567027.8,0.8,157020001.656000


In [6]:
#Set column widths for RPS
colspecs=[[0,14], [21,26], [46,55], [55,66]]
names=['Line Number', 'GS Number', 'Easting', 'Northing']

#Create a dataframe for the RPS
all_rps = pd.DataFrame()

for i in rps_files:
    df = pd.read_fwf(i,colspecs=colspecs,names=names)
    
    df = df.loc[df["Line Number"].str.startswith("R")]
    all_rps = all_rps.append(df) 

#Convert E/N to numbers 
all_rps["Easting"] = pd.to_numeric(all_rps["Easting"])
all_rps["Northing"] = pd.to_numeric(all_rps["Northing"])


#Check
all_rps.head()

,Line Number,GS Number,Easting,Northing
0,RPO18201182501,14661,462943.28,6559449.38
1,RPO18201182501,14681,462955.28,6559471.38
2,RPO18201182501,14701,462968.28,6559493.38
3,RPO18201182501,14721,462980.28,6559514.38
4,RPO18201182501,14741,462993.28,6559536.38


In [7]:
#Create geometries for sps
geometry = [Point(xy) for xy in zip(all_sps.Easting, all_sps.Northing)]
crs = {'init': 'epsg:23031'}

#Create points for sps
all_sps_points = GeoDataFrame(all_sps, crs=crs, geometry=geometry)

#Create lines for sps
all_sps_lines = all_sps_points.groupby(['Line Number'])['geometry'].apply(lambda x: LineString(x.tolist()))
all_sps_lines = GeoDataFrame(all_sps_lines, geometry='geometry')

#Check
all_sps_points.head()
all_sps_lines.head()

,geometry
Line Number,
S11779001,"LINESTRING (466667.6 6567071.6, 466684.6 65670..."
S11783004,"LINESTRING (466710.6 6567046.9, 466727.8 65670..."
S11787045,"LINESTRING (461273.1 6557542.4, 461302.6 65575..."
S11791047,"LINESTRING (461323.6 6557513.2, 461351.8 65575..."
S11795048,"LINESTRING (466833.7 6566975.9, 466850 6566951..."


In [8]:
#Create shapefiles for sps
all_sps_points.to_file(driver = 'ESRI Shapefile', filename= os.environ['USERPROFILE']+"\\"+output_path+"\\result_SPS_points.shp")
all_sps_lines.to_file(driver = 'ESRI Shapefile', filename= os.environ['USERPROFILE']+"\\"+output_path+"\\result_SPS_lines.shp")

In [9]:
#Create geometries for rps
geometry = [Point(xy) for xy in zip(all_rps.Easting, all_rps.Northing)]
crs = {'init': 'epsg:23031'}

#Create points for rps
all_rps_points = GeoDataFrame(all_rps, crs=crs, geometry=geometry)

#Create lines for rps
all_rps_lines = all_rps_points.groupby(['Line Number'])['geometry'].apply(lambda x: LineString(x.tolist()))
all_rps_lines = GeoDataFrame(all_rps_lines, geometry='geometry')

#Check
all_rps_points.head()
all_rps_lines.head()

,geometry
Line Number,
RPO18201182501,"LINESTRING (462943.28 6559449.38, 462955.28 65..."
RPO18201185701,"LINESTRING (463289.29 6559249.38, 463302.29 65..."
RPO18201188901,"LINESTRING (463635.89 6559049.68, 463648.39 65..."
RPO18201192101,"LINESTRING (463982.29 6558849.68, 463994.79 65..."


In [11]:
#Create shapefiles for rps
all_rps_points.to_file(driver = 'ESRI Shapefile', filename= os.environ['USERPROFILE']+"\\"+output_path+"\\result_RPS_points.shp")
all_rps_lines.to_file(driver = 'ESRI Shapefile', filename= os.environ['USERPROFILE']+"\\"+output_path+"\\result_RPS_lines.shp")